In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# --- Configuración ---
warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

print("Iniciando EDA para la Datathon...")

# --- 1. Carga y Vistazo Inicial ---
# NOTA: He usado 'train_sample.csv' (el archivo de muestra)
# Cambia esto a tu archivo 'train.csv' completo cuando lo ejecutes.
file_name = '../mango/train_sample.csv'
try:
    df = pd.read_csv(file_name, delimiter=';')
    print(f"Archivo '{file_name}' cargado exitosamente.")
    
    print("\n--- 1.1. Información General del DataFrame ---")
    df.info()

    print("\n--- 1.2. Primeras Filas (Head) ---")
    print(df.head())

    print("\n--- 1.3. Valores Nulos ---")
    # Imprimimos solo las columnas que SÍ tienen nulos para ser más concisos
    null_counts = df.isnull().sum()
    print(null_counts[null_counts > 0])

except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    # Salimos si no se puede cargar el archivo
    exit()

# --- 2. Análisis Nivel Producto (Features Estáticas - Fase 1 y 2) ---
# Estas son las features que definen el "QUÉ" y el "CONTEXTO"

print("\n--- 2.1. Cardinalidad (Valores Únicos) ---")
# Usamos try-except para cada print de cardinalidad por si alguna otra columna falta
try:
    print(f"Productos (ID) únicos: {df['ID'].nunique()}")
    print(f"Temporadas (id_season) únicas: {df['id_season'].nunique()}")
    print(f"Familias (family) únicas: {df['family'].nunique()}")
    print(f"Categorías (category) únicas: {df['category'].nunique()}")
    print(f"Siluetas (silhouette_type) únicas: {df['silhouette_type'].nunique()}")
    print(f"Momentos (moment) únicos: {df['moment'].nunique()}")
    
    # --- CORRECCIÓN ---
    # La columna 'ocassion' (con dos 's') no existe en la cabecera que proporcionaste.
    # La comentamos para evitar el 'KeyError'.
    # print(f"Ocasiones (ocassion) únicas: {df['ocassion'].nunique()}")
    
    # Verificamos si existe 'occasion' (con una 's')
    if 'occasion' in df.columns:
        print(f"Ocasiones (occasion) únicas: {df['occasion'].nunique()}")
    else:
        print("La columna 'ocassion' u 'occasion' no se encontró.")
        
except KeyError as e:
    print(f"Error de columna en Cardinalidad: {e}")


# --- 2.2. Distribución de Features Numéricas Estáticas (Contexto - Fase 2) ---
static_num_cols = ['num_stores', 'num_sizes', 'price', 'life_cycle_length']
product_level_static = df.drop_duplicates(subset=['ID'])

print(f"\n--- 2.3. Descriptivos de Features Numéricas (por Producto) ---")
existing_static_cols = [col for col in static_num_cols if col in product_level_static.columns]
if existing_static_cols:
    print(product_level_static[existing_static_cols].describe())
else:
    print("Ninguna de las columnas numéricas estáticas esperadas se encontró.")

# Graficamos sus distribuciones
if len(existing_static_cols) > 0:
    num_cols_plot = len(existing_static_cols)
    fig_cols = 2
    fig_rows = int(np.ceil(num_cols_plot / fig_cols))
    
    fig, axes = plt.subplots(fig_rows, fig_cols, figsize=(15, 5 * fig_rows))
    fig.suptitle('Distribución de Features Numéricas Estáticas (por Producto)', fontsize=16)
    
    axes = axes.flatten() 
    
    for i, col in enumerate(existing_static_cols):
        # Solo graficar si la columna tiene datos no nulos
        if not product_level_static[col].isnull().all():
            sns.histplot(product_level_static[col], ax=axes[i], kde=True)
            axes[i].set_title(f'Distribución de {col}')
        else:
            axes[i].set_title(f'Distribución de {col} (Vacío)')
            axes[i].set_visible(False)
        
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    #plt.savefig('static_features_distribution.png')
    print("Guardado: 'static_features_distribution.png'")
else:
    print("No se generó el gráfico de distribución, columnas no encontradas.")


# --- 3. Análisis Nivel Semana (Series Temporales - Fase 2) ---
print("\n--- 3.1. Descriptivos de Targets y Sales (por Semana) ---")
time_series_cols = ['weekly_sales', 'weekly_demand', 'Production']
existing_ts_cols = [col for col in time_series_cols if col in df.columns]
if existing_ts_cols:
    print(df[existing_ts_cols].describe())
else:
    print("Columnas de series temporales no encontradas.")

if 'ID' in df.columns and 'Production' in df.columns:
    production_variability = df.groupby('ID')['Production'].nunique().max()
    print(f"\nMáximo de valores de 'Production' distintos por ID: {production_variability}")
    if production_variability == 1:
        print("Confirmado: 'Production' es una feature estática (constante por ID).")
    elif production_variability == 0:
         print("Advertencia: No hay datos de 'Production' para verificar su variabilidad.")
    else:
        print("Advertencia: 'Production' varía para un mismo ID. Esto es inesperado.")
else:
    print("No se pudo verificar la variabilidad de 'Production'. Faltan 'ID' o 'Production'.")

if 'weekly_sales' in df.columns and 'Production' in df.columns and 'life_cycle_length' in df.columns:
    df_safe = df[df['life_cycle_length'] > 0].copy()
    if not df_safe.empty and not df_safe['Production'].isnull().all():
        df_safe['avg_weekly_prod'] = df_safe['Production'] / df_safe['life_cycle_length']
        df_safe['sales_capped_by_prod'] = df_safe['weekly_sales'] < df_safe['avg_weekly_prod']
        print("\nRelación Ventas vs Producción (simplificado, media semanal):")
        print(df_safe['sales_capped_by_prod'].value_counts(normalize=True))
    else:
        print("No se pudo calcular la relación ventas vs producción (datos insuficientes).")
else:
    print("No se pudo calcular la relación ventas vs producción. Faltan columnas.")


# --- 4. El Gráfico Clave: Producción vs. Demanda Total (Fase 4) ---
if 'ID' in df.columns and 'weekly_demand' in df.columns and 'weekly_sales' in df.columns and 'Production' in df.columns:
    product_summary = df.groupby('ID').agg(
        total_demand=('weekly_demand', 'sum'),
        total_sales=('weekly_sales', 'sum'),
        production=('Production', 'first') 
    )

    print("\n--- 4.1. Descriptivos de Totales (por Producto) ---")
    print(product_summary[['total_demand', 'total_sales', 'production']].describe())

    # --- CORRECCIÓN GRÁFICO ---
    # Solo intentar graficar si hay datos (product_summary no está vacío y las columnas no son todas NaN)
    if not product_summary.empty and not product_summary['production'].isnull().all() and not product_summary['total_demand'].isnull().all():
        plt.figure(figsize=(12, 8))
        sns.scatterplot(data=product_summary, x='production', y='total_demand', alpha=0.6, label='Productos')

        max_val = max(product_summary['production'].max(), product_summary['total_demand'].max())
        min_val = min(product_summary['production'].min(), product_summary['total_demand'].min())
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Producción = Demanda (Línea y=x)')

        plt.title('Demanda Total vs. Producción (Nivel Producto)', fontsize=16, pad=20)
        plt.xlabel('Producción (Lo que se compró)', fontsize=12)
        plt.ylabel('Demanda Total (Potencial de Venta)', fontsize=12)
        plt.legend()
        plt.text(max_val * 0.1, max_val * 0.9, 'Zona de Venta Perdida (Sub-producción)\n(production < total_demand)', color='red', fontsize=10, ha='left')
        plt.text(max_val * 0.9, max_val * 0.1, 'Zona de Exceso de Stock (Sobre-producción)\n(production > total_demand)', color='blue', fontsize=10, ha='right')

        #plt.savefig('production_vs_demand_scatter.png')
        print("Guardado: 'production_vs_demand_scatter.png'")
    else:
        print("No se generó el gráfico 'production_vs_demand_scatter.png', datos insuficientes en el archivo de muestra.")

else:
    print("No se pudo generar el resumen de producto. Faltan columnas clave (ID, weekly_demand, weekly_sales, Production).")


print("\n--- EDA Finalizado ---")